In [29]:
# 連結Google drive檔案
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)

file_path =("/content/EN_generated.xlsx")


Mounted at /content/drive


In [30]:
!pip install openpyxl

In [31]:
df = pd.read_excel(file_path)

# Calculate the total number of news content
total_news_content = len(df)

# Calculate unique news titles


# Impact Duration Distribution based on the exact values of 0, 1, and 2
impact_duration_distribution = df['Impact_Duration'].value_counts().sort_index()

# Impact Level Distribution


# Output the calculated values
print(f"Total News Content: {total_news_content}")

print("Impact Duration Distribution:")
print(impact_duration_distribution)

Total News Content: 7499
Impact Duration Distribution:
Impact_Duration
0    2497
1    2498
2    2504
Name: count, dtype: int64


In [32]:
!pip install transformers
!pip install sklearn

!pip install scikit-learn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import get_linear_schedule_with_warmup, AdamW
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [34]:
# Encode the 'impact_duration' labels to numeric values
label_encoder = LabelEncoder()
df['impact_duration_encoded'] = label_encoder.fit_transform(df['Impact_Duration'])
# Check for NaN values in 'news_content'
if df['news_content'].isnull().any():
    # Handle NaN values, e.g., by replacing them with a placeholder string
    df['news_content'].fillna('No content', inplace=True)

# Split the data into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_data(df):
    # Ensure all entries are strings and handle missing values if necessary
    texts = df['news_content'].astype(str).tolist()  # Convert to string to avoid issues
    return tokenizer(texts, truncation=True, padding=True, max_length=512)


train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

train_labels = train_df['impact_duration_encoded'].tolist()
val_labels = val_df['impact_duration_encoded'].tolist()

# Create a custom dataset for PyTorch
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_encoder.classes_))

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "l

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [36]:
!pip install transformers==4.26.0
!pip install accelerate==0.14.0
!pip install scikit-learn

In [37]:
import accelerate
print(accelerate.__version__)

0.14.0


In [38]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

# Compute metrics function for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    # Calculate weighted precision, recall, and F1
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted')

    # Calculate macro F1
    _, _, f1_macro, _ = precision_recall_fscore_support(labels, predictions, average='macro')

    # Calculate micro F1
    _, _, f1_micro, _ = precision_recall_fscore_support(labels, predictions, average='micro')

    # Calculate accuracy
    acc = accuracy_score(labels, predictions)

    # Return all metrics
    return {
        'accuracy': acc,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted
    }

# Setup optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=training_args.warmup_steps, num_training_steps=num_training_steps)

# Initialize the trainer with custom optimizer and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)  # Pass custom optimizer and scheduler
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()
print(evaluation_results)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5999
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2250
  Number of trainable parameters = 177855747


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Macro,F1 Micro,Precision Weighted,Recall Weighted
1,0.631200,0.417544,0.905333,0.904329,0.904054,0.905333,0.911947,0.905333
2,0.126300,0.124265,0.975333,0.975328,0.975232,0.975333,0.975725,0.975333
3,0.037700,0.091622,0.986667,0.986673,0.986590,0.986667,0.986731,0.986667


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16


Training completed. Do not forget to share your model on hugging

{'eval_loss': 0.0916222631931305, 'eval_accuracy': 0.9866666666666667, 'eval_f1_weighted': 0.986672741456197, 'eval_f1_macro': 0.9865898233813791, 'eval_f1_micro': 0.9866666666666668, 'eval_precision_weighted': 0.9867310168789377, 'eval_recall_weighted': 0.9866666666666667, 'eval_runtime': 4.9482, 'eval_samples_per_second': 303.143, 'eval_steps_per_second': 18.997, 'epoch': 3.0}


In [45]:
# Step 1: Load the JSON file
test_file_path = '/content/EN_Test.xlsx'  # Adjust the path accordingly
test_df = pd.read_excel(test_file_path)

# Calculate the total number of news content
total_news_content = len(test_df)

# Calculate unique news titles


# Impact Duration Distribution based on the exact values of 0, 1, and 2
impact_duration_distribution = df['Impact_Duration'].value_counts().sort_index()

# Impact Level Distribution


# Output the calculated values
print(f"Total News Content: {total_news_content}")

print("Impact Duration Distribution:")
print(impact_duration_distribution)




Total News Content: 136
Impact Duration Distribution:
Impact_Duration
0     6
1    47
2    83
Name: count, dtype: int64


In [50]:
# Convert the loaded data into a DataFrame (assuming it's a list of dictionaries)
test_df = pd.DataFrame(test_data)

# Step 2: Preprocess the test data
label_encoder = LabelEncoder()
test_df['impact_duration_encoded'] = label_encoder.fit_transform(test_df['Impact_Duration'])

if test_df['news_content'].isnull().any():
    # Handle NaN values, e.g., by replacing them with a placeholder string
    test_df['news_content'].fillna('No content', inplace=True)

# Encode labels using the same LabelEncoder used for training
#test_df['impact_duration_encoded'] = label_encoder.transform(test_df['Impact_Duration'])

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Tokenize the test data
test_encodings = tokenize_data(test_df)
test_labels = test_df['impact_duration_encoded'].tolist()

# Create a custom dataset for PyTorch
class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create a PyTorch dataset for the test data
test_dataset = NewsDataset(test_encodings, test_labels)


# Step 3: Evaluate the model on the test dataset
test_results = trainer.evaluate(test_dataset=test_dataset)
print("Test Results:", test_results)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/3f076fdb1ab68d5b2880cb87a0886f315b8146f8/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "l

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 41>:41                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Trainer.evaluate() got an unexpected keyword argument 'test_dataset'

In [ ]:

# Generate predictions for the training dataset
train_predictions = trainer.predict(train_dataset)

# Generate predictions for the validation dataset
val_predictions = trainer.predict(val_dataset)

# The predictions are in logits, so you need to apply softmax to convert to probabilities and then take the argmax to get the predicted labels
train_pred_labels = np.argmax(train_predictions.predictions, axis=1)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)

# The true labels are already provided as part of the datasets
train_true_labels = train_predictions.label_ids
val_true_labels = val_predictions.label_ids

# Calculate accuracy for the training dataset
train_accuracy = accuracy_score(train_true_labels, train_pred_labels)

# Calculate accuracy for the validation dataset
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)

# To print classification reports, you'll need to import it
from sklearn.metrics import classification_report

# Convert the label encoder's classes to strings for use as target names
target_names = [str(label) for label in label_encoder.inverse_transform(range(len(label_encoder.classes_)))]

# Then use these target names in your classification report
print("Training Dataset Metrics:")
print(f"Accuracy: {train_accuracy}")
print(classification_report(train_true_labels, train_pred_labels, target_names=target_names))

print("Validation Dataset Metrics:")
print(f"Accuracy: {val_accuracy}")
print(classification_report(val_true_labels, val_pred_labels, target_names=target_names))



Adam optimizer

In [ ]:
from transformers import get_linear_schedule_with_warmup, AdamW


# Example of setting up an optimizer with weight decay
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Example of setting up a learning rate scheduler
num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Update training_args to include gradient clipping
training_args.gradient_accumulation_steps = 1
training_args.max_grad_norm = 1.0

# Implementing a Trainer with custom optimizer and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler)  # Pass custom optimizer and scheduler
)

# Fine-tuning with the Trainer
trainer.train()


In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy for the training dataset
train_accuracy = accuracy_score(train_true_labels, train_pred_labels)

# Calculate accuracy for the validation dataset
val_accuracy = accuracy_score(val_true_labels, val_pred_labels)

# Print accuracy along with the classification report
print("Training Dataset Metrics:")
print(f"Accuracy: {train_accuracy}")
print(classification_report(train_true_labels, train_pred_labels))

print("Validation Dataset Metrics:")
print(f"Accuracy: {val_accuracy}")
# print(classification_report(val_true_labels, val_pred_labels))

In [ ]:
import pandas as pd

# Load the test dataset
test_dataset_path = '/content/ML-ESG-3_Test_output_Chinese.json'
test_df = pd.read_json(test_dataset_path)

# Display the first few rows of the test dataset to understand its structure
test_df.head()

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import Dataset
import torch
import numpy as np


# Assuming you have already loaded your model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the test dataset
def tokenize_data(df):
    texts = df['news_content'].astype(str).tolist()  # Ensure all entries are strings
    return tokenizer(texts, truncation=True, padding=True, max_length=512)

test_encodings = tokenize_data(test_df)

# Dummy labels for creating the Dataset object, since labels are not needed for prediction
test_labels = [0] * len(test_df)

class TestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

test_dataset = TestDataset(test_encodings)

# Predict using the trained model
predictions = trainer.predict(test_dataset)

# Convert predictions to labels
predicted_labels = np.argmax(predictions.predictions, axis=1)
original_labels = label_encoder.inverse_transform(predicted_labels)

# Add predictions to the test DataFrame
test_df['predicted_impact_duration'] = original_labels
import os
print(os.getcwd())

# Save the updated DataFrame to a new JSON file in the current working directory
output_filename = 'updated_test_dataset_with_predictions.json'
test_df.to_json(output_filename, orient='records', lines=True)

print(f"Updated test dataset saved to {output_filename}")
